In [3]:
import os
import pandas as pd
import numpy as np
import sys
import os
from common import commons
home = commons.home
logger = commons.logger
extra_storage = commons.extra_storage
dataset = commons.dataset
from features_preprocess import BED_binning
from features_preprocess import BED_Preprocess, CADD_Preprocess,DANN_Preprocess,Eigen_Preprocess,GenoCanyon_Preprocess,WGBS_preprocess,GWAVA_Preprocess
import subprocess
from features_preprocess import get_winid
from common.commons import rename_features,check_genocaynon
import os
import re
import gc

INFO    ; 2019-06-02 04:38:25,285; root           ; commons:<module>;39: Existing logger at commons


In [4]:

def nearest_tss(tss,sites_df):
    merged = pd.merge(sites_df,tss,how='outer',on=['chr','coordinate'])
    merged.sort_values(['chr','coordinate'],inplace=True)
    merged.rename(columns={'strand':'before_tss'},inplace=True)
    merged.loc[merged['before_tss'].isnull()==False, 'before_tss'] = merged.loc[merged['before_tss'].isnull()==False,'coordinate']
    merged['after_tss'] = merged['before_tss']
    merged['before_tss'].fillna(method='ffill', inplace=True)
    merged['after_tss'].fillna(method='bfill',inplace=True)
    merged['dist_to_before_tss'] = np.abs(merged['coordinate']-merged['before_tss'])
    merged['dist_to_after_tss'] = np.abs(merged['coordinate']-merged['after_tss'])
    merged['tss'] = None
    before_ix = (merged['dist_to_before_tss'] < merged['dist_to_after_tss']) | (merged['dist_to_after_tss'].isnull())
    merged.loc[before_ix,'tss'] = merged.loc[before_ix,'before_tss']
    after_ix = (merged['dist_to_before_tss'] >= merged['dist_to_after_tss']) | (merged['dist_to_before_tss'].isnull())
    merged.loc[after_ix,'tss'] = merged.loc[after_ix,'after_tss']
    merged['dist_to_nearest_tss'] = np.abs(merged['coordinate']-merged['tss']) 
    merged.drop(['before_tss','after_tss','tss','dist_to_before_tss','dist_to_after_tss'],axis=1,inplace=True)
    merged.dropna(axis=0,subset=['id'],inplace=True)
    return merged

In [15]:
if dataset == 'AD_CpG':
    type_name = 'cerad'  ## amyloid, cerad, tangles
    with_cell_type = commons.with_cell_type ## with or without
    dataset = dataset+'/'+type_name+with_cell_type
sites_file = home+'data/'+dataset+'/all_sites_winid.csv'
additional_feature_file = home+'data/features/'+dataset+'/addtional_features'

In [4]:
logger.info('Concatening features to all training sites of {}'.format(dataset))
logger.info('Read all training sites file of {} with window id at {}'.format(dataset,sites_file))
logger.info("Additional features (WGBS,ATAC,etc) are saved at {}".format(additional_feature_file))

INFO    ; 2019-06-01 13:28:24,750; root           ; <ipython-input-4-45e173830dc8>:<module>;1: Concatening features to all training sites of AD_CpG/braakwith
INFO    ; 2019-06-01 13:28:24,751; root           ; <ipython-input-4-45e173830dc8>:<module>;2: Read all training sites file of AD_CpG/braakwith with window id at /home/ec2-user/xiaobo/git/EnsembleCpG/data/AD_CpG/braakwith/all_sites_winid.csv
INFO    ; 2019-06-01 13:28:24,752; root           ; <ipython-input-4-45e173830dc8>:<module>;3: Additional features (WGBS,ATAC,etc) are saved at /home/ec2-user/xiaobo/git/EnsembleCpG/data/features/AD_CpG/braakwith/addtional_features


In [18]:
###processed features
with pd.HDFStore(home+'data/'+dataset+'/all_features','r') as h5s:
     all_sites = h5s['all_features']

In [19]:
all_sites

,id,chr,coordinate,beta_sign,pvalue,beta,label,winid,Breast_vHMEC,CD14_Primary_Cells,...,NCFF843SYR_WGBS_counts,NCFF847OWL_WGBS_counts,NCFF874GGB_WGBS_counts,NCFF913ZNZ_WGBS_counts,NCFF923CZC_WGBS_counts,GWAVA_region_score,GWAVA_tss_score,GWAVA_unmatched_score,GWAVA_dist_to_nearest_snp,dist_to_nearest_tss
0,cg19724344,1,877647.0,-0.708163,4.790807e-01,0.107180,0.0,4389.0,7.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.27,0.45,0.96,39.0,1936.0
1,cg22381068,1,906363.0,-0.663568,5.071860e-01,0.241956,0.0,4532.0,1.0,4.0,...,1.0,3.0,6.0,0.0,0.0,0.33,0.36,0.80,1.0,4216.0
2,cg26003967,1,933987.0,-0.526639,5.986115e-01,0.199789,0.0,4670.0,9.0,6.0,...,0.0,6.0,0.0,0.0,0.0,0.28,0.52,0.85,15.0,355.0
3,cg06538872,1,947343.0,-0.728290,4.666810e-01,0.721224,0.0,4737.0,4.0,4.0,...,41.0,22.0,56.0,39.0,17.0,0.25,0.18,0.18,50.0,1504.0
4,cg13362546,1,1102960.0,-0.282153,7.779102e-01,0.780087,0.0,5515.0,4.0,6.0,...,7.0,7.0,6.0,2.0,1.0,0.42,0.46,0.55,75.0,283.0
5,cg14531564,1,1154853.0,-0.566441,5.712760e-01,0.968517,0.0,5775.0,3.0,5.0,...,7.0,9.0,8.0,9.0,10.0,0.14,0.07,0.44,5.0,2565.0
6,cg21265287,1,1228383.0,4.490833,8.301720e-06,0.638471,1.0,6142.0,3.0,1.0,...,17.0,8.0,8.0,1.0,2.0,0.47,0.71,0.83,57.0,619.0
7,cg17132079,1,1294140.0,-0.172371,8.631960e-01,0.492184,0.0,6471.0,7.0,11.0,...,0.0,4.0,6.0,1.0,0.0,0.27,0.31,0.78,103.0,6069.0
8,cg01536190,1,1473147.0,-0.054911,9.562251e-01,0.739637,0.0,7366.0,3.0,4.0,...,24.0,21.0,23.0,13.0,9.0,0.31,0.21,0.47,10.0,2989.0
9,cg06912282,1,1563001.0,-0.806752,4.200843e-01,0.765561,0.0,7816.0,6.0,10.0,...,3.0,6.0,13.0,1.0,0.0,0.22,0.39,0.85,1.0,4559.0


In [13]:
all_sites.query('winid==14403481')[['A549-CTCF','A549-H2A','A549-H3K4me1','A549-H3K4me2','A549-H3K4me3','A549-H3K9ac']]

,A549-CTCF,A549-H2A,A549-H3K4me1,A549-H3K4me2,A549-H3K4me3,A549-H3K9ac
2085,90.0,35.0,18.0,159.0,272.0,157.0


In [115]:
dataset

'AD_CpG/ceradwith'

In [9]:
home

'/home/ec2-user/xiaobo/git/EnsembleCpG/'

In [10]:
extra_storage

'/home/ec2-user/xiaobo/git/EnsembleCpG/data/raw/'

In [103]:
additional_feature_file

'/home/ec2-user/xiaobo/git/EnsembleCpG/data/features/AD_CpG/ceradwith/addtional_features'

In [16]:
with pd.HDFStore(additional_feature_file,'r') as h5s:
    df = h5s['GenoCanyon'] 

In [17]:
df

,chr,coordinate,winid,genocanyon_score
0,1,877647.0,4389,0.936024
1,1,906363.0,4532,0.630474
2,1,933987.0,4670,0.887939
3,1,947343.0,4737,0.805571
4,1,1102960.0,5515,0.994697
5,1,1154853.0,5775,0.624586
6,1,1228383.0,6142,0.524341
7,1,1294140.0,6471,0.963801
8,1,1473147.0,7366,0.695281
9,1,1563001.0,7816,0.745616


In [108]:
all_sites.query('chr==22 and (coordinate==50640274.0)').filter(regex='genocanyon_score')

,genocanyon_score
2472,0.999751


In [5]:
#check all_450k is consistent with selected training sites
all_450_features = home+'data/'+commons.dataset+'/all_450k_features' 
with pd.HDFStore(all_450_features,'r') as h5s:
    all_450_sites = h5s['all_450k_features'] 

In [53]:
all_450_sites

,id,chr,coordinate,winid,Breast_vHMEC,CD14_Primary_Cells,CD19_Primary_Cells,CD20_Primary_Cells,CD3_Primary_Cells,CD4_Primary_Cells,...,NCFF843SYR_WGBS_counts,NCFF847OWL_WGBS_counts,NCFF874GGB_WGBS_counts,NCFF913ZNZ_WGBS_counts,NCFF923CZC_WGBS_counts,GWAVA_region_score,GWAVA_tss_score,GWAVA_unmatched_score,GWAVA_dist_to_nearest_snp,dist_to_nearest_tss
0,cg13869341,1,15865.0,80.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,11.0,3.0,0.0,0.62,0.27,0.58,45.0,1503.0
1,cg15560884,1,710097.0,3551.0,0.0,0.0,3.0,1.0,1.0,5.0,...,83.0,38.0,18.0,62.0,21.0,0.43,0.14,0.15,82.0,9852.0
2,cg01014490,1,714177.0,3571.0,487.0,779.0,1202.0,177.0,1476.0,2846.0,...,0.0,0.0,0.0,0.0,0.0,0.60,0.35,0.93,12.0,13932.0
3,cg17505339,1,720865.0,3605.0,0.0,1.0,4.0,2.0,2.0,1.0,...,35.0,23.0,30.0,21.0,16.0,0.48,0.10,0.25,0.0,20620.0
4,cg16736630,1,779995.0,3900.0,2.0,1.0,5.0,0.0,24.0,42.0,...,16.0,26.0,32.0,15.0,9.0,0.40,0.18,0.37,32.0,17024.0
5,cg03128332,1,805338.0,4027.0,16.0,98.0,288.0,18.0,281.0,590.0,...,0.0,0.0,0.0,0.0,0.0,0.56,0.48,0.82,160.0,1887.0
6,cg18147296,1,812539.0,4063.0,2.0,5.0,12.0,3.0,5.0,14.0,...,12.0,0.0,5.0,15.0,6.0,0.39,0.27,0.60,13.0,9088.0
7,cg05597748,1,812679.0,4064.0,0.0,6.0,7.0,4.0,4.0,6.0,...,20.0,0.0,16.0,4.0,3.0,0.47,0.19,0.56,0.0,9228.0
8,cg10988531,1,813317.0,4067.0,1.0,5.0,7.0,0.0,2.0,4.0,...,25.0,0.0,4.0,23.0,6.0,0.45,0.47,0.59,0.0,9866.0
9,cg13938959,1,834183.0,4171.0,14.0,3.0,8.0,1.0,8.0,22.0,...,18.0,20.0,14.0,21.0,19.0,0.43,0.33,0.28,15.0,18770.0


In [85]:
all_sites

,id,chr,coordinate,beta_sign,pvalue,beta,label,winid,Breast_vHMEC,CD14_Primary_Cells,...,NCFF843SYR_WGBS_counts,NCFF847OWL_WGBS_counts,NCFF874GGB_WGBS_counts,NCFF913ZNZ_WGBS_counts,NCFF923CZC_WGBS_counts,GWAVA_region_score,GWAVA_tss_score,GWAVA_unmatched_score,GWAVA_dist_to_nearest_snp,dist_to_nearest_tss
0,cg22381068,1,906363.0,0.002513,0.960036,0.241956,0.0,4532.0,1.0,4.0,...,1.0,3.0,6.0,0.0,0.0,0.330,0.36,0.800,1.0,4216.0
1,cg24730157,1,984428.0,0.704868,0.401441,0.919376,0.0,4923.0,1.0,5.0,...,2.0,7.0,1.0,2.0,1.0,0.200,0.23,0.530,2.0,22698.0
2,cg08480609,1,986560.0,0.011082,0.916192,0.783207,0.0,4933.0,3.0,1.0,...,4.0,15.0,6.0,8.0,5.0,0.310,0.05,0.300,43.0,20566.0
3,cg04972348,1,1100035.0,17.611301,0.000031,0.643412,1.0,5501.0,1.0,3.0,...,4.0,4.0,3.0,0.0,0.0,0.390,0.55,0.600,33.0,2449.0
4,cg14144728,1,1101462.0,0.157120,0.691943,0.788976,0.0,5508.0,8.0,8.0,...,3.0,0.0,1.0,7.0,3.0,0.340,0.55,0.680,7.0,1022.0
5,cg08733913,1,1190778.0,0.556278,0.456015,0.944233,0.0,5954.0,2.0,5.0,...,10.0,16.0,20.0,7.0,9.0,0.540,0.42,0.710,1.0,1486.0
6,cg24555071,1,1358016.0,0.570040,0.450498,0.594937,0.0,6791.0,14.0,3.0,...,5.0,21.0,30.0,4.0,4.0,0.390,0.25,0.710,23.0,3492.0
7,cg24470133,1,1483206.0,0.176690,0.674363,0.960068,0.0,7417.0,1.0,11.0,...,16.0,6.0,22.0,4.0,15.0,0.280,0.06,0.160,19.0,6153.0
8,cg22373622,1,1489544.0,21.924244,0.000003,0.918745,1.0,7448.0,6.0,6.0,...,22.0,8.0,15.0,7.0,10.0,0.250,0.18,0.180,6.0,12491.0
9,cg15198247,1,1974844.0,0.448279,0.503375,0.677556,0.0,9875.0,1.0,3.0,...,5.0,16.0,12.0,4.0,6.0,0.410,0.40,0.230,1.0,7065.0


In [27]:
selected_pos_450K_merged = pd.merge(all_sites,all_450_sites,on=['id'],how='left')
    

In [28]:
selected_pos_450K_merged.query('genocanyon_score_x != genocanyon_score_y')[['chr_x','coordinate_x','coordinate_y','genocanyon_score_x','genocanyon_score_y']]

,chr_x,coordinate_x,coordinate_y,genocanyon_score_x,genocanyon_score_y


In [120]:
all_sites.query('chr==1 and (coordinate==933987)')['genocanyon_score']

2    0.805571
Name: genocanyon_score, dtype: float64

In [36]:
all_sites

,id,chr,coordinate,beta_sign,pvalue,beta,label,winid,Breast_vHMEC,CD14_Primary_Cells,...,NCFF843SYR_WGBS_counts,NCFF847OWL_WGBS_counts,NCFF874GGB_WGBS_counts,NCFF913ZNZ_WGBS_counts,NCFF923CZC_WGBS_counts,GWAVA_region_score,GWAVA_tss_score,GWAVA_unmatched_score,GWAVA_dist_to_nearest_snp,dist_to_nearest_tss
0,cg14144728,1,1101462.0,0.753117,0.451636,0.788976,0.0,5508.0,8.0,8.0,...,3.0,0.0,1.0,7.0,3.0,0.34,0.55,0.68,7.0,1022.0
1,cg13362546,1,1102960.0,0.010134,0.991917,0.780087,0.0,5515.0,4.0,6.0,...,7.0,7.0,6.0,2.0,1.0,0.42,0.46,0.55,75.0,283.0
2,cg01937247,1,1229534.0,-0.472933,0.636410,0.761588,0.0,6148.0,2.0,8.0,...,9.0,11.0,7.0,8.0,10.0,0.29,0.41,0.75,4.0,1770.0
3,cg22389936,1,1328033.0,-0.011136,0.991118,0.860997,0.0,6641.0,3.0,4.0,...,21.0,35.0,56.0,29.0,12.0,0.59,0.39,0.59,163.0,6877.0
4,cg24470133,1,1483206.0,0.038365,0.969408,0.960068,0.0,7417.0,1.0,11.0,...,16.0,6.0,22.0,4.0,15.0,0.28,0.06,0.16,19.0,6153.0
5,cg06912282,1,1563001.0,0.533732,0.593699,0.765561,0.0,7816.0,6.0,10.0,...,3.0,6.0,13.0,1.0,0.0,0.22,0.39,0.85,1.0,4559.0
6,cg21221161,1,1999183.0,0.262879,0.792723,0.878506,0.0,9996.0,3.0,3.0,...,31.0,20.0,24.0,30.0,17.0,0.12,0.15,0.19,20.0,17274.0
7,cg10334053,1,2078117.0,0.094347,0.924861,0.802265,0.0,10391.0,4.0,6.0,...,18.0,34.0,19.0,13.0,7.0,0.19,0.16,0.24,0.0,37782.0
8,cg10769649,1,2111330.0,0.020466,0.983678,0.762031,0.0,10557.0,6.0,5.0,...,30.0,17.0,10.0,20.0,24.0,0.20,0.28,0.29,13.0,4569.0
9,cg16545954,1,2118288.0,-0.230976,0.817402,0.761599,0.0,10592.0,5.0,4.0,...,34.0,19.0,40.0,19.0,18.0,0.45,0.49,0.79,1.0,2389.0


In [34]:
cols = all_sites.columns[8:]

In [35]:
cols

Index(['Breast_vHMEC', 'CD14_Primary_Cells', 'CD19_Primary_Cells',
       'CD20_Primary_Cells', 'CD3_Primary_Cells', 'CD4_Primary_Cells',
       'CD56_Primary_Cells', 'CD8_Primary_Cells', 'Fetal_Adrenal_Gland',
       'Fetal_Brain',
       ...
       'NCFF843SYR_WGBS_counts', 'NCFF847OWL_WGBS_counts',
       'NCFF874GGB_WGBS_counts', 'NCFF913ZNZ_WGBS_counts',
       'NCFF923CZC_WGBS_counts', 'GWAVA_region_score', 'GWAVA_tss_score',
       'GWAVA_unmatched_score', 'GWAVA_dist_to_nearest_snp',
       'dist_to_nearest_tss'],
      dtype='object', length=2256)

In [84]:
selected_pos_450K_merged

,id,chr_x,coordinate_x,beta_sign,pvalue,beta,label,winid_x,Breast_vHMEC_x,CD14_Primary_Cells_x,...,NCFF843SYR_WGBS_counts_y,NCFF847OWL_WGBS_counts_y,NCFF874GGB_WGBS_counts_y,NCFF913ZNZ_WGBS_counts_y,NCFF923CZC_WGBS_counts_y,GWAVA_region_score_y,GWAVA_tss_score_y,GWAVA_unmatched_score_y,GWAVA_dist_to_nearest_snp_y,dist_to_nearest_tss_y
0,cg22381068,1,906363.0,0.002513,0.960036,0.241956,0.0,4532.0,1.0,4.0,...,1.0,3.0,6.0,0.0,0.0,0.330,0.36,0.800,1.0,4216.0
1,cg24730157,1,984428.0,0.704868,0.401441,0.919376,0.0,4923.0,1.0,5.0,...,2.0,7.0,1.0,2.0,1.0,0.200,0.23,0.530,2.0,22698.0
2,cg08480609,1,986560.0,0.011082,0.916192,0.783207,0.0,4933.0,3.0,1.0,...,4.0,15.0,6.0,8.0,5.0,0.310,0.05,0.300,43.0,20566.0
3,cg04972348,1,1100035.0,17.611301,0.000031,0.643412,1.0,5501.0,1.0,3.0,...,4.0,4.0,3.0,0.0,0.0,0.390,0.55,0.600,33.0,2449.0
4,cg14144728,1,1101462.0,0.157120,0.691943,0.788976,0.0,5508.0,8.0,8.0,...,3.0,0.0,1.0,7.0,3.0,0.340,0.55,0.680,7.0,1022.0
5,cg08733913,1,1190778.0,0.556278,0.456015,0.944233,0.0,5954.0,2.0,5.0,...,10.0,16.0,20.0,7.0,9.0,0.540,0.42,0.710,1.0,1486.0
6,cg24555071,1,1358016.0,0.570040,0.450498,0.594937,0.0,6791.0,14.0,3.0,...,5.0,21.0,30.0,4.0,4.0,0.390,0.25,0.710,23.0,3492.0
7,cg24470133,1,1483206.0,0.176690,0.674363,0.960068,0.0,7417.0,1.0,11.0,...,16.0,6.0,22.0,4.0,15.0,0.280,0.06,0.160,19.0,6153.0
8,cg22373622,1,1489544.0,21.924244,0.000003,0.918745,1.0,7448.0,6.0,6.0,...,22.0,8.0,15.0,7.0,10.0,0.250,0.18,0.180,6.0,12491.0
9,cg15198247,1,1974844.0,0.448279,0.503375,0.677556,0.0,9875.0,1.0,3.0,...,5.0,16.0,12.0,4.0,6.0,0.410,0.40,0.230,1.0,7065.0


In [ ]:
with pd.HDFStore(home+'data/'+dataset+'/all_features','r') as h5s:
     all_sites = h5s['all_features']

In [31]:
datasets = ['AD_CpG/amyloidwith','AD_CpG/ceradwith','AD_CpG/ceradafwith','AD_CpG/tangleswith','AD_CpG/gpathwith','AD_CpG/braakwith','AD_CpG/cogdecwith']

In [37]:
for dataset in datasets:
    with pd.HDFStore(home+'data/'+dataset+'/all_features','r') as h5s:
         all_sites = h5s['all_features']
    print(f'dataset is {dataset}')
    selected_pos_450K_merged = pd.merge(all_sites,all_450_sites,on=['id'],how='left')
    print('merged')
    for col in cols:
        if not selected_pos_450K_merged[f'{col}_x'].equals(selected_pos_450K_merged[f'{col}_y']):
            print(f'{dataset}:{col}')
    print('{dataset} check completed')

dataset is AD_CpG/amyloidwith
merged
{dataset} check completed
dataset is AD_CpG/ceradwith
merged
{dataset} check completed
dataset is AD_CpG/ceradafwith
merged
{dataset} check completed
dataset is AD_CpG/tangleswith
merged
{dataset} check completed
dataset is AD_CpG/gpathwith
merged
{dataset} check completed
dataset is AD_CpG/braakwith
merged
{dataset} check completed
dataset is AD_CpG/cogdecwith
merged
{dataset} check completed


In [94]:
selected_pos_450K_merged = pd.merge(all_sites,all_450_sites,on=['id'],how='left')

In [47]:
tss_end = 26573858

In [2]:
#check all_450k is consistent with all WGBS sites
from prediction.prediction_commons import *

In [6]:
ranges = np.arange(tss_start,tss_end,2000000)
ranges = np.append(ranges,tss_end)

In [7]:
ranges

array([       0,  2000000,  4000000,  6000000,  8000000, 10000000,
       12000000, 14000000, 16000000, 18000000, 20000000, 22000000,
       24000000, 26000000, 26573858])

In [8]:
cols = all_450_sites.columns[4:]

In [9]:
import gc

In [11]:
for i in np.arange(len(ranges)-1):

    start = ranges[i]
    end = ranges[i+1]
    logger.info("start processing range %d to %d"%(start,end))
    all_features = home+'data/WGBS/all_features_'+str(start)+'_'+str(end)
    
    with pd.HDFStore(all_features,'r') as h5s:
        logger.info('Get sites with all features within range {} to {} from {}'.format(start,end,all_features))
        wgbs_all_data = h5s['all_features']
    print(f'read wgbs all features {start}-{end}')
    wgbs_all_data = pd.merge(all_450_sites,wgbs_all_data,on=['chr','coordinate'],how='inner')
    print("merging finished")
    for col in cols:
        if not wgbs_all_data[f'{col}_x'].equals(wgbs_all_data[f'{col}_y']):
            print(f'{start}-{end}:{col}')
    print(f'{start}-{end} check completed')
    del wgbs_all_data
    gc.collect()    

INFO    ; 2019-06-02 04:39:50,651; root           ; <ipython-input-11-25968606c626>:<module>;5: start processing range 0 to 2000000
INFO    ; 2019-06-02 04:39:50,655; root           ; <ipython-input-11-25968606c626>:<module>;9: Get sites with all features within range 0 to 2000000 from /home/ec2-user/xiaobo/git/EnsembleCpG/data/WGBS/all_features_0_2000000


read wgbs all features 0-2000000
merging finished


INFO    ; 2019-06-02 04:45:52,525; root           ; <ipython-input-11-25968606c626>:<module>;5: start processing range 2000000 to 4000000
INFO    ; 2019-06-02 04:45:52,528; root           ; <ipython-input-11-25968606c626>:<module>;9: Get sites with all features within range 2000000 to 4000000 from /home/ec2-user/xiaobo/git/EnsembleCpG/data/WGBS/all_features_2000000_4000000


0-2000000 check completed
read wgbs all features 2000000-4000000
merging finished


INFO    ; 2019-06-02 04:51:50,947; root           ; <ipython-input-11-25968606c626>:<module>;5: start processing range 4000000 to 6000000
INFO    ; 2019-06-02 04:51:50,951; root           ; <ipython-input-11-25968606c626>:<module>;9: Get sites with all features within range 4000000 to 6000000 from /home/ec2-user/xiaobo/git/EnsembleCpG/data/WGBS/all_features_4000000_6000000


...

20000000-22000000 check completed
read wgbs all features 22000000-24000000
merging finished


INFO    ; 2019-06-02 05:51:26,259; root           ; <ipython-input-11-25968606c626>:<module>;5: start processing range 24000000 to 26000000
INFO    ; 2019-06-02 05:51:26,262; root           ; <ipython-input-11-25968606c626>:<module>;9: Get sites with all features within range 24000000 to 26000000 from /home/ec2-user/xiaobo/git/EnsembleCpG/data/WGBS/all_features_24000000_26000000


22000000-24000000 check completed
read wgbs all features 24000000-26000000
merging finished


INFO    ; 2019-06-02 05:57:24,896; root           ; <ipython-input-11-25968606c626>:<module>;5: start processing range 26000000 to 26573858
INFO    ; 2019-06-02 05:57:24,898; root           ; <ipython-input-11-25968606c626>:<module>;9: Get sites with all features within range 26000000 to 26573858 from /home/ec2-user/xiaobo/git/EnsembleCpG/data/WGBS/all_features_26000000_26573858


24000000-26000000 check completed
read wgbs all features 26000000-26573858
merging finished
26000000-26573858 check completed


In [54]:
merged = pd.merge(all_450_sites,wgbs_all_data,on=['chr','coordinate'],how='inner')

In [55]:
merged

,id,chr,coordinate,winid_x,Breast_vHMEC_x,CD14_Primary_Cells_x,CD19_Primary_Cells_x,CD20_Primary_Cells_x,CD3_Primary_Cells_x,CD4_Primary_Cells_x,...,NCFF843SYR_WGBS_counts_y,NCFF847OWL_WGBS_counts_y,NCFF874GGB_WGBS_counts_y,NCFF913ZNZ_WGBS_counts_y,NCFF923CZC_WGBS_counts_y,GWAVA_region_score_y,GWAVA_tss_score_y,GWAVA_unmatched_score_y,GWAVA_dist_to_nearest_snp_y,dist_to_nearest_tss_y
0,cg13869341,1,15865.0,80.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,11.0,3.0,0.0,0.62,0.27,0.58,45.0,1503.0
1,cg15560884,1,710097.0,3551.0,0.0,0.0,3.0,1.0,1.0,5.0,...,83.0,38.0,18.0,62.0,21.0,0.43,0.14,0.15,82.0,9852.0
2,cg01014490,1,714177.0,3571.0,487.0,779.0,1202.0,177.0,1476.0,2846.0,...,0.0,0.0,0.0,0.0,0.0,0.60,0.35,0.93,12.0,13932.0
3,cg17505339,1,720865.0,3605.0,0.0,1.0,4.0,2.0,2.0,1.0,...,35.0,23.0,30.0,21.0,16.0,0.48,0.10,0.25,0.0,20620.0
4,cg16736630,1,779995.0,3900.0,2.0,1.0,5.0,0.0,24.0,42.0,...,16.0,26.0,32.0,15.0,9.0,0.40,0.18,0.37,32.0,17024.0
5,cg03128332,1,805338.0,4027.0,16.0,98.0,288.0,18.0,281.0,590.0,...,0.0,0.0,0.0,0.0,0.0,0.56,0.48,0.82,160.0,1887.0
6,cg18147296,1,812539.0,4063.0,2.0,5.0,12.0,3.0,5.0,14.0,...,12.0,0.0,5.0,15.0,6.0,0.39,0.27,0.60,13.0,9088.0
7,cg05597748,1,812679.0,4064.0,0.0,6.0,7.0,4.0,4.0,6.0,...,20.0,0.0,16.0,4.0,3.0,0.47,0.19,0.56,0.0,9228.0
8,cg10988531,1,813317.0,4067.0,1.0,5.0,7.0,0.0,2.0,4.0,...,25.0,0.0,4.0,23.0,6.0,0.45,0.47,0.59,0.0,9866.0
9,cg13938959,1,834183.0,4171.0,14.0,3.0,8.0,1.0,8.0,22.0,...,18.0,20.0,14.0,21.0,19.0,0.43,0.33,0.28,15.0,18770.0


In [58]:
for col in cols:
    if not merged[f'{col}_x'].equals(merged[f'{col}_y']):
        print(f'{col}')